# Transforming Infractions into Inspections

In [1]:
%load_ext lab_black
%load_ext autoreload
%autoreload 2

In [2]:
import configparser
from datetime import datetime
from typing import Union

import pandas as pd
import snowflake.connector

## About

In this notebook, we will transform the raw DineSafe data from infractions into inspections. The raw data has multiple rows (infractions) for each inspection. We will need to condense this into a single row per inspection.

We will also filter out unwanted inspections based on various criteria discussed later in this notebook.

## User Inputs

In [3]:
transformed_fname_prefix = "filtered_transformed_data"

ci_run = "no"

In [4]:
if ci_run == "yes":
    ACCOUNT = os.getenv("SNOWFLAKE_ACCOUNT")
    USER = os.getenv("SNOWFLAKE_USER")
    PASS = os.getenv("SNOWFLAKE_PASS")
    WAREHOUSE = os.getenv("SNOWFLAKE_WAREHOUSE")
    DB_SCHEMA = os.getenv("SNOWFLAKE_DB_SCHEMA")
    DB_NAME = "dinesafe"
else:
    config = configparser.ConfigParser()
    config.read("../sql.ini")
    default_cfg = config["default"]
    ACCOUNT = default_cfg["SNOWFLAKE_ACCOUNT"]
    USER = default_cfg["SNOWFLAKE_USER"]
    PASS = default_cfg["SNOWFLAKE_PASS"]
    WAREHOUSE = default_cfg["SNOWFLAKE_WAREHOUSE"]
    DB_SCHEMA = default_cfg["SNOWFLAKE_DB_SCHEMA"]
    DB_NAME = "dinesafe"

In [5]:
connector_dict = dict(
    account=ACCOUNT,
    user=USER,
    password=PASS,
    database=DB_NAME,
    schema="public",
    warehouse=WAREHOUSE,
    role="sysadmin",
)

In [6]:
def show_sql_df(
    query: str,
    cursor,
    cnx=None,
    table_output: bool = False,
    use_manual_approach: bool = False,
    show_df: bool = True,
) -> Union[None, pd.DataFrame]:
    cursor.execute(query)
    if cnx:
        cnx.commit()
    if table_output:
        if use_manual_approach:
            colnames = [cdesc[0].lower() for cdesc in cursor.description]
            cur_fetched = cursor.fetchall()
            if cur_fetched:
                df_query_output = pd.DataFrame.from_records(
                    cur_fetched, columns=colnames, index=range(len(cur_fetched))
                )
                if show_df:
                    display(df_query_output)
                return df_query_output
        else:
            df_query_output = cursor.fetch_pandas_all().reset_index(drop=True)
            if show_df:
                display(df_query_output)
            return df_query_output
    return pd.DataFrame()

## Connect to SQL Database

In [7]:
conn = snowflake.connector.connect(**connector_dict)
cur = conn.cursor()

## Preliminary Exploration of Data

In [8]:
%%time
query = """
        SELECT *
        FROM inspections
        LIMIT 6
        """
_ = show_sql_df(query, cur, None, True, False)

,ROW_ID,ESTABLISHMENT_ID,INSPECTION_ID,ESTABLISHMENT_NAME,ESTABLISHMENTTYPE,ESTABLISHMENT_ADDRESS,ESTABLISHMENT_STATUS,MINIMUM_INSPECTIONS_PERYEAR,INFRACTION_DETAILS,INSPECTION_DATE,SEVERITY,ACTION,COURT_OUTCOME,AMOUNT_FINED,LATITUDE,LONGITUDE
0,1,1222579,102810896,SAI-LILA KHAMAN DHOKLA HOUSE,Food Take Out,870 MARKHAM RD,Pass,2,None,2012-08-21,None,None,None,NaN,NaN,NaN
1,2,1222579,103015258,SAI-LILA KHAMAN DHOKLA HOUSE,Food Take Out,870 MARKHAM RD,Conditional Pass,2,FAIL TO PROVIDE TOWELS IN FOOD PREPARATION ARE...,2013-06-26,S - Significant,Corrected During Inspection,None,NaN,NaN,NaN
2,3,1222579,103015258,SAI-LILA KHAMAN DHOKLA HOUSE,Food Take Out,870 MARKHAM RD,Conditional Pass,2,Food handler fail to wear headgear,2013-06-26,M - Minor,Notice to Comply,None,NaN,NaN,NaN
3,4,1222579,103015258,SAI-LILA KHAMAN DHOKLA HOUSE,Food Take Out,870 MARKHAM RD,Conditional Pass,2,Operator fail to ensure food is not contaminat...,2013-06-26,C - Crucial,Notice to Comply,None,NaN,NaN,NaN
4,5,1222579,103015258,SAI-LILA KHAMAN DHOKLA HOUSE,Food Take Out,870 MARKHAM RD,Conditional Pass,2,Operator fail to maintain hazardous food(s) at...,2013-06-26,C - Crucial,Notice to Comply,None,NaN,NaN,NaN
5,6,1222579,103015258,SAI-LILA KHAMAN DHOKLA HOUSE,Food Take Out,870 MARKHAM RD,Conditional Pass,2,Operator fail to properly maintain rooms,2013-06-26,M - Minor,Notice to Comply,None,NaN,NaN,NaN


CPU times: user 18.6 ms, sys: 5.67 ms, total: 24.2 ms
Wall time: 712 ms


### Count number of establishments per Inspection

Count the number of establishments per inspection (we should have, at most, one restaurant for each inspection ID) and sort the results in descending order of the number of establishments

In [9]:
%%time
query = """
        SELECT inspection_id,
               COUNT(DISTINCT(establishment_id)) AS num_establishments
        FROM inspections
        GROUP BY inspection_id
        ORDER BY COUNT(DISTINCT(inspection_id)) DESC
        """
_ = show_sql_df(query, cur, None, True, False)

,INSPECTION_ID,NUM_ESTABLISHMENTS
0,104229741,1
1,103347971,1
2,104175560,1
3,103346510,1
4,103386413,1
...,...,...
246629,102913263,1
246630,102704120,1
246631,102855605,1
246632,102574240,1


CPU times: user 440 ms, sys: 54.6 ms, total: 495 ms
Wall time: 1.55 s


The results are sorted in descending order, so the maximum number of establishments per inspection appear first. As expected, we only have one establishment recorded per inspection ID.

### Count number of Inspections per Establishment

Count the number of inspections per establishment (we should have one or more inspections per establishment)

In [10]:
%%time
query = """
        SELECT establishment_id,
               establishmenttype,
               establishment_address,
               COUNT(DISTINCT(inspection_id)) AS num_inspections
        FROM inspections
        GROUP BY establishment_id, establishmenttype, establishment_address
        ORDER BY COUNT(DISTINCT(inspection_id)) DESC
        """
_ = show_sql_df(query, cur, None, True, False)

,ESTABLISHMENT_ID,ESTABLISHMENTTYPE,ESTABLISHMENT_ADDRESS,NUM_INSPECTIONS
0,10336522,Supermarket,4466 SHEPPARD AVE E,60
1,10282501,Bakery,2300 LAWRENCE AVE E,51
2,9011824,Restaurant,4386 SHEPPARD AVE E,48
3,10399527,Food Take Out,4810 SHEPPARD AVE E,48
4,10420908,Restaurant,3601 VICTORIA PARK AVE,47
...,...,...,...,...
30285,10597238,Restaurant,456 QUEEN ST W,1
30286,10652164,Food Store (Convenience / Variety),2150 BLOOR ST W,1
30287,9012452,Food Store (Convenience / Variety),522 UNIVERSITY AVE,1
30288,10588519,Restaurant,817 KIPLING AVE,1


CPU times: user 26.1 ms, sys: 13.1 ms, total: 39.1 ms
Wall time: 1.36 s


We do have one or more inspections per establishment inspected. Since an establishment can be inspected multiple times, this is expected.

### Count number of infractions per inspection

To do this, it only makes sense to count the number of infractions per inspection per establishment

In [11]:
%%time
query = """
        SELECT establishment_id,
               establishmenttype,
               establishment_address,
               inspection_id,
               COUNT(*) AS number_of_infractions
        FROM inspections
        GROUP BY establishment_id, establishmenttype, establishment_address, inspection_id
        ORDER BY COUNT(*) DESC
        """
_ = show_sql_df(query, cur, None, True, False)

,ESTABLISHMENT_ID,ESTABLISHMENTTYPE,ESTABLISHMENT_ADDRESS,INSPECTION_ID,NUMBER_OF_INFRACTIONS
0,10356286,Restaurant,4016 FINCH AVE E,103465643,90
1,9031081,Food Take Out,200 WELLINGTON ST W,103580745,84
2,10528444,Food Processing Plant,19 WATERMAN AVE,103473708,84
3,10522734,Restaurant,1686 ELLESMERE RD,103428956,80
4,10191833,Restaurant,5594 YONGE ST,103598378,80
...,...,...,...,...,...
247323,10339687,Restaurant,3730 KINGSTON RD,102764882,1
247324,10339703,Food Take Out,300 BOROUGH DR,102579977,1
247325,10339794,Food Store (Convenience / Variety),2389 YONGE ST,102679323,1
247326,10341035,Bakery,1969 QUEEN ST E,102872729,1


CPU times: user 412 ms, sys: 33 ms, total: 445 ms
Wall time: 1.97 s


As we can see, there can be one or more infractions per inspection performed at a given establishment.

The first inspection found above did indeed detect 90 infractions as shown below

In [12]:
%%time
query = """
        SELECT *
        FROM inspections
        WHERE inspection_id = 103465643
        """
_ = show_sql_df(query, cur, None, True, False)

,ROW_ID,ESTABLISHMENT_ID,INSPECTION_ID,ESTABLISHMENT_NAME,ESTABLISHMENTTYPE,ESTABLISHMENT_ADDRESS,ESTABLISHMENT_STATUS,MINIMUM_INSPECTIONS_PERYEAR,INFRACTION_DETAILS,INSPECTION_DATE,SEVERITY,ACTION,COURT_OUTCOME,AMOUNT_FINED,LATITUDE,LONGITUDE
0,35165,10356286,103465643,MILLIKEN BAR RESTAURANT,Restaurant,4016 FINCH AVE E,Conditional Pass,3,Operator fail to clean washroom fixtures,2015-04-15,S - Significant,Notice to Comply,None,NaN,NaN,NaN
1,35166,10356286,103465643,MILLIKEN BAR RESTAURANT,Restaurant,4016 FINCH AVE E,Conditional Pass,3,Operator fail to ensure food is not contaminat...,2015-04-15,C - Crucial,Notice to Comply,None,NaN,NaN,NaN
2,35167,10356286,103465643,MILLIKEN BAR RESTAURANT,Restaurant,4016 FINCH AVE E,Conditional Pass,3,Operator fail to maintain hazardous foods at 6...,2015-04-15,C - Crucial,Notice to Comply,None,NaN,NaN,NaN
3,35168,10356286,103465643,MILLIKEN BAR RESTAURANT,Restaurant,4016 FINCH AVE E,Conditional Pass,3,Operator fail to properly maintain equipment,2015-04-15,S - Significant,Notice to Comply,None,NaN,NaN,NaN
4,35169,10356286,103465643,MILLIKEN BAR RESTAURANT,Restaurant,4016 FINCH AVE E,Conditional Pass,3,Operator fail to properly maintain equipment(N...,2015-04-15,M - Minor,Notice to Comply,None,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85,51245,10356286,103465643,MILLIKEN BAR RESTAURANT,Restaurant,4016 FINCH AVE E,Conditional Pass,3,Operator fail to provide hand washing supplies,2015-04-15,S - Significant,Notice to Comply,None,NaN,NaN,NaN
86,51246,10356286,103465643,MILLIKEN BAR RESTAURANT,Restaurant,4016 FINCH AVE E,Conditional Pass,3,Operator fail to provide proper equipment,2015-04-15,M - Minor,Notice to Comply,None,NaN,NaN,NaN
87,51247,10356286,103465643,MILLIKEN BAR RESTAURANT,Restaurant,4016 FINCH AVE E,Conditional Pass,3,Operator fail to provide proper garbage contai...,2015-04-15,M - Minor,Notice to Comply,None,NaN,NaN,NaN
88,51248,10356286,103465643,MILLIKEN BAR RESTAURANT,Restaurant,4016 FINCH AVE E,Conditional Pass,3,Operator fail to use proper procedure(s) to en...,2015-04-15,S - Significant,Notice to Comply,None,NaN,NaN,NaN


CPU times: user 19 ms, sys: 0 ns, total: 19 ms
Wall time: 368 ms


### Get Data with a Missing Address

In [13]:
%%time
query = """
        SELECT COUNT(*) AS num_missing_addresses
        FROM inspections
        WHERE establishment_address IS NULL
        """
df_miss_address = show_sql_df(query, cur, None, True, False, False)
df_miss_address.head()

CPU times: user 5.42 ms, sys: 0 ns, total: 5.42 ms
Wall time: 792 ms


,NUM_MISSING_ADDRESSES
0,0


There are no inspections where the establishment address is missing.

### Get Data with a Missing Latitude or Longitude

Get all inspections missing either a latitude or longitude
- in the next notebook, we will geocode these locations so that we can (later) determine the neighbourhood for each establishment and get supplementary datasets that provide metadata for each neighbourhood

In [14]:
%%time
query = """
        SELECT *
        FROM inspections
        WHERE latitude IS NULL
        OR longitude IS NULL
        """
df_miss_lat_lon = show_sql_df(query, cur, None, True, False, False)
df_miss_lat_lon.head()

CPU times: user 862 ms, sys: 161 ms, total: 1.02 s
Wall time: 2.59 s


,ROW_ID,ESTABLISHMENT_ID,INSPECTION_ID,ESTABLISHMENT_NAME,ESTABLISHMENTTYPE,ESTABLISHMENT_ADDRESS,ESTABLISHMENT_STATUS,MINIMUM_INSPECTIONS_PERYEAR,INFRACTION_DETAILS,INSPECTION_DATE,SEVERITY,ACTION,COURT_OUTCOME,AMOUNT_FINED,LATITUDE,LONGITUDE
0,1,1222579,102810896,SAI-LILA KHAMAN DHOKLA HOUSE,Food Take Out,870 MARKHAM RD,Pass,2,None,2012-08-21,None,None,None,NaN,NaN,NaN
1,2,1222579,103015258,SAI-LILA KHAMAN DHOKLA HOUSE,Food Take Out,870 MARKHAM RD,Conditional Pass,2,FAIL TO PROVIDE TOWELS IN FOOD PREPARATION ARE...,2013-06-26,S - Significant,Corrected During Inspection,None,NaN,NaN,NaN
2,3,1222579,103015258,SAI-LILA KHAMAN DHOKLA HOUSE,Food Take Out,870 MARKHAM RD,Conditional Pass,2,Food handler fail to wear headgear,2013-06-26,M - Minor,Notice to Comply,None,NaN,NaN,NaN
3,4,1222579,103015258,SAI-LILA KHAMAN DHOKLA HOUSE,Food Take Out,870 MARKHAM RD,Conditional Pass,2,Operator fail to ensure food is not contaminat...,2013-06-26,C - Crucial,Notice to Comply,None,NaN,NaN,NaN
4,5,1222579,103015258,SAI-LILA KHAMAN DHOKLA HOUSE,Food Take Out,870 MARKHAM RD,Conditional Pass,2,Operator fail to maintain hazardous food(s) at...,2013-06-26,C - Crucial,Notice to Comply,None,NaN,NaN,NaN


There are many rows with missing values in the `latitude` or `longitude` columns for the same address. As seen earlier, there could be many infractions recorded in this dataset for a single establishment (address) on a given date.

For geocoding purposes, we only need to get each (unique) address (with missing latitude or longitude) once. We'll now write a SQL query to give this output

In [15]:
%%time
query = """
        SELECT establishment_address,
               MAX(latitude) AS latitude,
               MAX(longitude) AS longitude
        FROM inspections
        WHERE latitude IS NULL
        OR longitude IS NULL
        GROUP BY establishment_address
        """
df_addr_lat_lon = show_sql_df(query, cur, None, True, False)

,ESTABLISHMENT_ADDRESS,LATITUDE,LONGITUDE
0,1867 YONGE ST,NaN,NaN
1,938 COLLEGE ST,NaN,NaN
2,800 DANFORTH AVE,NaN,NaN
3,681 MARKHAM RD,NaN,NaN
4,31 PASSMORE AVE,NaN,NaN
...,...,...,...
13280,2025 KIPLING AVE,NaN,NaN
13281,821 BLOOR ST W,NaN,NaN
13282,411 DANFORTH AVE,NaN,NaN
13283,1050 ALBION RD,NaN,NaN


CPU times: user 14.8 ms, sys: 3.46 ms, total: 18.2 ms
Wall time: 409 ms


Check that every row in the above query has missing values in **both** the `latitude` and `longitude` columns. To do this, we'll count the number of
- establishments
- missing values in the `latitude` column
- missing values in the `longitude` column

If each establishment with a missing value in the `latitude` or `longitude` column is **also** missing a value in the `longitude` or `latitude` column, then we will know that every establishment listed in the above query is missing values in **both** the `latitude` and `longitude` columns

In [16]:
%%time
query = """
        SELECT SUM(CASE WHEN latitude IS NULL THEN 1 ELSE 0 END) AS num_miss_lat,
               SUM(CASE WHEN longitude IS NULL THEN 1 ELSE 0 END) AS num_miss_lon,
               COUNT(DISTINCT(establishment_address)) AS num_establishments
        FROM (
            SELECT establishment_address,
                   MAX(latitude) AS latitude,
                   MAX(longitude) AS longitude
            FROM inspections
            WHERE latitude IS NULL
            OR longitude IS NULL
            GROUP BY establishment_address
        ) AS combo
        """
_ = show_sql_df(query, cur, None, True, False)

,NUM_MISS_LAT,NUM_MISS_LON,NUM_ESTABLISHMENTS
0,13285,13285,13285


CPU times: user 7.58 ms, sys: 0 ns, total: 7.58 ms
Wall time: 224 ms


Since these row counts agree with eachother, we can say that every establishment with a missing value in the `latitude` column is also missing a value in the `longitude` column, and vice-versa.

### Types of Infraction Severities

In [17]:
%%time
query = """
        SELECT severity,
               COUNT(*) AS num_infractions
        FROM inspections
        GROUP BY severity
        ORDER BY COUNT(*) DESC
        """
_ = show_sql_df(query, cur, None, True, False)

,SEVERITY,NUM_INFRACTIONS
0,None,359119
1,M - Minor,330971
2,S - Significant,223491
3,NA - Not Applicable,41503
4,C - Crucial,26715


CPU times: user 6.29 ms, sys: 787 µs, total: 7.08 ms
Wall time: 192 ms


**Observatoins**
1. Our model needs to be trained on inspections where the infraction details are known. It will need to know whether an infraction is significant / crucial or not. We may not need to keep inspections where the infraction severity is `NA - Not Applicable`, but we'll need to explore this next.

Show the infractions with a severity of `NA - ...`

In [18]:
%%time
query = """
        SELECT infraction_details,
               severity,
               establishment_status
        FROM inspections
        WHERE severity LIKE '%%NA -%%'
        """
with pd.option_context('display.max_colwidth',1000):
    _ = show_sql_df(query, cur, None, True, False)

,INFRACTION_DETAILS,SEVERITY,ESTABLISHMENT_STATUS
0,Operator must ensure that the eating or drinking establishment license is continually posted adjacent to the food safety inspection notice. Muncipal Code Chapter 545 Sec. 5G(4),NA - Not Applicable,Pass
1,Fail to produce valid food handler's certificate - Muncipal Code Chapter 545 Sec. 5G(17)(c),NA - Not Applicable,Pass
2,Fail to hold a valid food handler's certificate - Muncipal Code Chapter 545 Sec. 5G(17)(b),NA - Not Applicable,Pass
3,Operator must ensure to provide the Food Safety Inspection Report(s) relating to the currently posted food safety inspection notice when requested by any person. Muncipal Code Chapter 545 Sec. 5G(5),NA - Not Applicable,Pass
4,Fail to ensure the presence of the holder of a valid food handler's certificate - Muncipal Code Chapter 545 Sec. 5G(17)(a),NA - Not Applicable,Pass
...,...,...,...
41498,Fail to Ensure the Presence of the Holder of a Valid Food Handlers Certificate - Sec. 545- 157E(1 7)(a),NA - Not Applicable,Pass
41499,Fail to Ensure the Presence of the Holder of a Valid Food Handler's Certificate. Muncipal Code Chapter 545-157(17)(a),NA - Not Applicable,Pass
41500,Fail to Ensure the Presence of the Holder of a Valid Food Handler's Certificate. Muncipal Code Chapter 545-157(17)(a),NA - Not Applicable,Pass
41501,Fail to Ensure the Presence of the Holder of a Valid Food Handler's Certificate. Muncipal Code Chapter 545-157(17)(a),NA - Not Applicable,Pass


CPU times: user 83 ms, sys: 12.4 ms, total: 95.3 ms
Wall time: 889 ms


Show the assigned establishment status for infractions assigned a severity of `NA - ...`

In [19]:
%%time
query = """
        SELECT establishment_status,
               COUNT(*) AS num_rows
        FROM inspections
        WHERE severity LIKE '%%NA -%%'
        GROUP BY establishment_status
        """
with pd.option_context('display.max_colwidth',1000):
    _ = show_sql_df(query, cur, None, True, False)

,ESTABLISHMENT_STATUS,NUM_ROWS
0,Closed,124
1,Pass,31783
2,Conditional Pass,9596


CPU times: user 6.85 ms, sys: 977 µs, total: 7.83 ms
Wall time: 417 ms


**Observations**
1. Nearly all these infractions result in a `Pass` being assigned to th establishment. However, the infraction details column does suggest that some infraction was detected by the inspector. Unfortunately, there is no valid entry (Crucial, Significant or Minor) in the `severity` column for these inspections.
2. Could we map the `establishment_status` that is `Pass` or `Conditional Pass` to non-critical/significant infractions and `Closed` to critical/significant? If we can do this, then we would be justified in keeping these infractions (where severity is `NA - ...`); if not then we will have to drop them.

Show the infraction details for infractions with a severity of `NA - ...` that resulted in the establishment being `Closed`

In [20]:
%%time
query = """
        SELECT infraction_details,
               severity,
               establishment_status
        FROM inspections
        WHERE severity LIKE '%%NA -%%'
        AND establishment_status = 'Closed'
        """
with pd.option_context('display.max_colwidth',1000):
    _ = show_sql_df(query, cur, None, True, False)

,INFRACTION_DETAILS,SEVERITY,ESTABLISHMENT_STATUS
0,Fail to ensure the presence of the holder of a valid food handler's certificate - Muncipal Code Chapter 545 Sec. 5G(17)(a),NA - Not Applicable,Closed
1,Fail to ensure the presence of the holder of a valid food handler's certificate - Muncipal Code Chapter 545 Sec. 5G(17)(a),NA - Not Applicable,Closed
2,Fail to ensure the presence of the holder of a valid food handler's certificate - Muncipal Code Chapter 545 Sec. 5G(17)(a),NA - Not Applicable,Closed
3,Operator must ensure to provide the Food Safety Inspection Report(s) relating to the currently posted food safety inspection notice when requested by any person. Muncipal Code Chapter 545 Sec. 5G(5),NA - Not Applicable,Closed
4,Fail to ensure the presence of the holder of a valid food handler's certificate - Muncipal Code Chapter 545 Sec. 5G(17)(a),NA - Not Applicable,Closed
...,...,...,...
119,Fail to ensure the presence of the holder of a valid food handler's certificate - Muncipal Code Chapter 545 Sec. 5G(17)(a),NA - Not Applicable,Closed
120,Operator must ensure that the eating or drinking establishment license is continually posted adjacent to the food safety inspection notice. Muncipal Code Chapter 545 Sec. 5G(4),NA - Not Applicable,Closed
121,"Operator must ensure that the most recent food safety inspection notice, as issued without alteration, is continually and clearly posted at or near the entrance of the premises. Municipal Code 545 Sec. 5G(3)",NA - Not Applicable,Closed
122,Fail to ensure the presence of the holder of a valid food handler's certificate - Muncipal Code Chapter 545 Sec. 5G(17)(a),NA - Not Applicable,Closed


CPU times: user 9.23 ms, sys: 0 ns, total: 9.23 ms
Wall time: 294 ms


**Observations**
1. These seem like valid infractions that led to the establishment being closed. Unfortunately, we don't have a valid entry in the severity column. If the infraction was strong enough, then why did the inspector not assign a `Significant` or `Crucial` severity to the infraction? There seems to be some reasoning / judgement that was used in arriving at this conclusion that is not present in the Dinesafe dataset. Our ML model will not be able to learn from such inspections. So, we'll exclude infractions with such a severity (`NA - ...`) from the data.

Below, we show the number of infractions by severity (including those where there was no infraction) and the assigned establishment status

In [21]:
%%time
query = """
        SELECT severity,
               establishment_status,
               COUNT(*) AS num_infractions
        FROM inspections
        GROUP BY severity, establishment_status
        ORDER BY COUNT(*) DESC
        """
with pd.option_context('display.max_colwidth',1000):
    _ = show_sql_df(query, cur, None, True, False)

,SEVERITY,ESTABLISHMENT_STATUS,NUM_INFRACTIONS
0,None,Pass,359118
1,M - Minor,Pass,270153
2,S - Significant,Pass,125361
3,S - Significant,Conditional Pass,96556
4,M - Minor,Conditional Pass,59629
5,NA - Not Applicable,Pass,31783
6,C - Crucial,Conditional Pass,25475
7,NA - Not Applicable,Conditional Pass,9596
8,S - Significant,Closed,1574
9,C - Crucial,Closed,1197


CPU times: user 6.71 ms, sys: 1.47 ms, total: 8.18 ms
Wall time: 191 ms


**Observations**
1. An establishment status of `Pass` or `Conditional Pass` could be associated with a crucial severity, not just minor, significant or `N/A- ...`. Similarly, `Closed` is associated with a minor and crucial severity. So, we cannot map `Pass` and `Conditional Pass` to non-significant / crucical severity and `Closed` to significant / crucial respectively. To eliminate this uncertainty in how to handle such inspections, we must drop inspections where the severity contains `NA - ...`.

### Select from the Different Types of Establishments that were Inspected

The number of inspections and infractions (including non-infractions) is shown for each type of establishment below

In [22]:
%%time
query = """
        SELECT establishmenttype,
               COUNT(DISTINCT(inspection_id)) AS num_inspections,
               COUNT(infraction_details) AS num_infractions
        FROM inspections
        WHERE severity NOT LIKE '%%NA -' AND severity IS NOT NULL
        GROUP BY establishmenttype
        ORDER BY COUNT(DISTINCT(inspection_id)) DESC
        """
with pd.option_context('display.max_colwidth',1000):
    _ = show_sql_df(query, cur, None, True, False)

,ESTABLISHMENTTYPE,NUM_INSPECTIONS,NUM_INFRACTIONS
0,Restaurant,55410,379246
1,Food Take Out,13460,82247
2,Food Store (Convenience / Variety),4009,22724
3,Food Court Vendor,3622,22896
4,Supermarket,3416,27161
5,Bakery,2621,20501
6,Butcher Shop,971,6749
7,Child Care - Food Preparation,928,3875
8,Child Care - Catered,919,3868
9,Food Caterer,864,5435


CPU times: user 8.59 ms, sys: 102 µs, total: 8.7 ms
Wall time: 295 ms


We will keep the following establishment types since they are equivalent to a restaurant (retail food establishments) or grocery stores, which has been used by other cities in ML applications with inspections data ([1](https://www.researchgate.net/publication/336418044_Identification_of_critical_factors_for_assessing_the_quality_of_restaurants_using_data_mining_approaches), [2](https://www.nature.com/articles/s41746-018-0045-1.pdf), [3](https://chicago.github.io/food-inspections-evaluation/))

In [23]:
establishment_types_wanted = [
    "Restaurant",
    "Food Take Out",
    "Food Store (Convenience / Variety)",  # equivalent to grocery store
    "Food Court Vendor",
    "Supermarket",  # equivalent to grocery store
    "Bakery",  # equivalent to grocery store
    # "Food Caterer",
    "Butcher Shop",  # equivalent to grocery store
    "Cafeteria - Public Access",
    # "Boarding / Lodging Home - Kitchen",
    "Cocktail Bar / Beverage Room",
    # "Food Depot",
    # "Private Club",
    "Fish Shop",  # equivalent to grocery store
    "Bake Shop",  # equivalent to grocery store
    # "Food Bank",
    "Flea Market",  # equivalent to grocery store
    "Farmer\\'s Market",  # equivalent to grocery store
    # "Bed & Breakfast",
]

The following is a string of all these wanted establishment types joined together, so that it can be used as a SQL filter (in the `WHERE` clause)

In [24]:
establishment_types_wanted_str = "('" + "', '".join(establishment_types_wanted) + "')"
print(establishment_types_wanted_str)

('Restaurant', 'Food Take Out', 'Food Store (Convenience / Variety)', 'Food Court Vendor', 'Supermarket', 'Bakery', 'Butcher Shop', 'Cafeteria - Public Access', 'Cocktail Bar / Beverage Room', 'Fish Shop', 'Bake Shop', 'Flea Market', 'Farmer\'s Market')


**Observations**
1. The following are a subset of the non-retail food establishments that were inspected
   - private / non-public
     - boarding / lodging home - kitchen
     - Private Club
     - Bed & Breakfast
   - niche (similar to school cafeteria)
     - food depot
     - food bank

   and these are distinct from restaurants and grocery stores, so they are excluded from analysis here.
2. Schools, private establishment (private club, etc.) and Hospitals do not present a risk that can be generalized across the population of the city like restaurants and grocery stores. Also, these two types of establishments follow a different inspection and planning protocol.

## Removing Invalid Data

We will now prepare the dataset using the above filters. To do this, we will want one inspection per row (observation), as mentioned earlier. So, we will group by each establishment and inspection and then aggregate (count) the number of each type of infraction (a zero is assigned if no infractions were recorded). These aggregated columns will be used to create the class labels (whether a significant / crucial infraction occurred or not).

### Aggregate Data to get one inspection per row

Count the number of each type of infraction recorded during a single inspection. Also, combine all the (text) details of each infraction (`infraction_details`) into an `infractions_summary` column.

In [25]:
%%time
query = f"""
        SELECT establishment_id,
               establishmenttype,
               establishment_address,
               inspection_date,
               inspection_id,
               -- GROUP_CONCAT(infraction_details SEPARATOR '. ') AS infractions_summary,
               LISTAGG(infraction_details, '. ') AS infractions_summary,
               establishment_status,
               action,
               court_outcome,
               SUM(CASE WHEN severity LIKE '%%S - Significant' THEN 1 ELSE 0 END) AS num_significant,
               SUM(CASE WHEN severity LIKE '%%C - Crucial' THEN 1 ELSE 0 END) AS num_crucial,
               SUM(CASE WHEN severity LIKE '%%M - Minor' THEN 1 ELSE 0 END) AS num_minor,
               SUM(CASE WHEN severity LIKE '%%NA -' THEN 1 ELSE 0 END) AS num_na,
               COUNT(infraction_details) AS num_infractions
        FROM inspections
        WHERE establishmenttype IN {establishment_types_wanted_str}
        AND severity IS NULL
        OR severity IN ('S - Significant', 'C - Crucial', 'M - Minor')
        GROUP BY establishment_id, establishmenttype, establishment_address, inspection_date, inspection_id, establishment_status, action, court_outcome
        """
df_query = show_sql_df(query, cur, None, True, False)

,ESTABLISHMENT_ID,ESTABLISHMENTTYPE,ESTABLISHMENT_ADDRESS,INSPECTION_DATE,INSPECTION_ID,INFRACTIONS_SUMMARY,ESTABLISHMENT_STATUS,ACTION,COURT_OUTCOME,NUM_SIGNIFICANT,NUM_CRUCIAL,NUM_MINOR,NUM_NA,NUM_INFRACTIONS
0,10275396,Hot Dog Cart,170 PRINCES BLVD,2015-05-10,103497075,IMMERSE UTENSILS IN CHLORINE SOLUTION OF LESS ...,Pass,Corrected During Inspection,None,5,0,0,0,5
1,10278238,Restaurant,207 OSSINGTON AVE,2015-04-30,103478543,Food handler fail to wear headgear. Food handl...,Pass,Notice to Comply,None,0,0,6,0,6
2,10282501,Bakery,2300 LAWRENCE AVE E,2014-09-18,103333486,Operator fail to maintain hazardous foods at 6...,Conditional Pass,Summons,None,0,4,0,0,4
3,10279215,Supermarket,3030 DON MILLS RD E,2015-02-25,103434089,USE EQUIPMENT NOT FREE FROM OPEN SEAMS IN CONT...,Pass,Corrected During Inspection,None,5,0,0,0,5
4,10282549,Cafeteria - Public Access,1881 YONGE ST,2015-04-30,103478039,Operator fail to properly wash equipment. Oper...,Pass,Notice to Comply,None,0,0,12,0,12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
238411,10420910,Restaurant,1921 LAWRENCE AVE E,2017-05-03,103960637,,Pass,None,None,0,0,0,0,0
238412,10554630,Restaurant,1630 BAYVIEW AVE,2018-07-05,104256412,FAIL TO REMOVE WASTES WHEN NECESSARY TO MAINTA...,Pass,Corrected During Inspection,None,1,0,0,0,1
238413,10557002,Food Take Out,101 PETER ST,2018-07-11,104260122,USE FOOD EQUIPMENT NOT OF READILY CLEANABLE FO...,Pass,Corrected During Inspection,None,1,0,0,0,1
238414,10459014,Food Take Out,777 BAY ST,2017-04-03,103942426,FAIL TO PROVIDE SOAP OR DETERGENT IN FOOD PREP...,Pass,Corrected During Inspection,None,2,0,0,0,2


CPU times: user 581 ms, sys: 91.3 ms, total: 673 ms
Wall time: 2.09 s


**Notes**
1. Multiple `action`s and `court_outcome`s are associated with a single inspection so each inspection currently covers multiple rows. So, a pivot table is necessary so that a single row has a single inspection. To do this, we will have to group by each establishment and inspection and
   - add up the number of each type of infraction
   - combine all the infraction details into a single string
   - move the `action` and `court_outcome` column from rows to columns
     - the aggregation should be to `SUM` the actions so that we have the total occurrences of a particular action and court outcome for each inspection

Use Python to create SQL needed to convert above query into a pivot table with establishments and inspections on the rows and all other fields as columns

In [26]:
%%time
query = """
        SELECT DISTINCT(action)
        FROM inspections
        """
df_actions = show_sql_df(query, cur, None, True, False, False)
display(df_actions)
action_strs = []
dtypes_dict = {}
for idx, row in df_actions.iterrows():
    act_str = row['ACTION'] if row['ACTION'] else 'NULL'
    action_value = f"= '{act_str}'" if act_str != 'NULL' else 'IS NULL'
    sql_str = f"SUM(CASE WHEN action {action_value} THEN 1 ELSE 0 END) AS num_{act_str.lower().replace(' ', '_')}"
    print(act_str, sql_str)
    action_strs.append(sql_str)
    dtypes_dict[f"num_{act_str.lower().replace(' ', '_')}"] = int
action_strs = ',\n'.join(action_strs)

,ACTION
0,Ticket
1,Corrected During Inspection
2,Summons and Health Hazard Order
3,Education Provided
4,Closure Order
5,Prohibition Order Requested
6,Not in Compliance
7,Notice to Comply
8,Summons
9,Order


Ticket SUM(CASE WHEN action = 'Ticket' THEN 1 ELSE 0 END) AS num_ticket
Corrected During Inspection SUM(CASE WHEN action = 'Corrected During Inspection' THEN 1 ELSE 0 END) AS num_corrected_during_inspection
Summons and Health Hazard Order SUM(CASE WHEN action = 'Summons and Health Hazard Order' THEN 1 ELSE 0 END) AS num_summons_and_health_hazard_order
Education Provided SUM(CASE WHEN action = 'Education Provided' THEN 1 ELSE 0 END) AS num_education_provided
Closure Order SUM(CASE WHEN action = 'Closure Order' THEN 1 ELSE 0 END) AS num_closure_order
Prohibition Order Requested SUM(CASE WHEN action = 'Prohibition Order Requested' THEN 1 ELSE 0 END) AS num_prohibition_order_requested
Not in Compliance SUM(CASE WHEN action = 'Not in Compliance' THEN 1 ELSE 0 END) AS num_not_in_compliance
Notice to Comply SUM(CASE WHEN action = 'Notice to Comply' THEN 1 ELSE 0 END) AS num_notice_to_comply
Summons SUM(CASE WHEN action = 'Summons' THEN 1 ELSE 0 END) AS num_summons
Order SUM(CASE WHEN action =

In [27]:
%%time
query = """
        SELECT DISTINCT(court_outcome)
        FROM inspections
        """
df_court_outcomes = show_sql_df(query, cur, None, True, False, False)
display(df_court_outcomes)
outcome_strs = []
outcomes_dtypes_dict = {}
for idx, row in df_court_outcomes.iterrows():
    outcome_str = row['COURT_OUTCOME'] if row['COURT_OUTCOME'] else 'NULL'
    outcome_value = f"= '{outcome_str}'" if outcome_str != 'NULL' else 'IS NULL'
    outcome_sql_str = (
        f"SUM(CASE WHEN court_outcome {outcome_value} THEN 1 ELSE 0 END) AS "
        f"num_{outcome_str.lower().replace(' ', '_').replace('-','').replace('__','_').replace('&_', '')}"
    )
    print(outcome_str, outcome_sql_str)
    outcome_strs.append(outcome_sql_str)
    outcomes_dtypes_dict[
        (
            f"num_{outcome_str.lower().replace(' ', '_').replace('-','').replace('__','_').replace('&_', '')}"
        )
    ] = int
outcome_strs = ',\n'.join(outcome_strs)

,COURT_OUTCOME
0,Charges Quashed
1,Conviction - Ordered to Close by Court
2,Charges Withdrawn
3,Pending
4,Cancelled
5,Conviction - Suspended Sentence
6,Conviction - Fined
7,Charges Dismissed
8,Conviction - Fined & Order to Close by Court
9,None


Charges Quashed SUM(CASE WHEN court_outcome = 'Charges Quashed' THEN 1 ELSE 0 END) AS num_charges_quashed
Conviction - Ordered to Close by Court SUM(CASE WHEN court_outcome = 'Conviction - Ordered to Close by Court' THEN 1 ELSE 0 END) AS num_conviction_ordered_to_close_by_court
Charges Withdrawn SUM(CASE WHEN court_outcome = 'Charges Withdrawn' THEN 1 ELSE 0 END) AS num_charges_withdrawn
Pending SUM(CASE WHEN court_outcome = 'Pending' THEN 1 ELSE 0 END) AS num_pending
Cancelled SUM(CASE WHEN court_outcome = 'Cancelled' THEN 1 ELSE 0 END) AS num_cancelled
Conviction - Suspended Sentence SUM(CASE WHEN court_outcome = 'Conviction - Suspended Sentence' THEN 1 ELSE 0 END) AS num_conviction_suspended_sentence
Conviction - Fined SUM(CASE WHEN court_outcome = 'Conviction - Fined' THEN 1 ELSE 0 END) AS num_conviction_fined
Charges Dismissed SUM(CASE WHEN court_outcome = 'Charges Dismissed' THEN 1 ELSE 0 END) AS num_charges_dismissed
Conviction - Fined & Order to Close by Court SUM(CASE WHEN cou

Convert above query into pivot table

In [28]:
%%time
query = f"""
        SELECT establishment_id,
               establishmenttype,
               establishment_address,
               inspection_date,
               inspection_id,
               establishment_status,
               -- GROUP_CONCAT(infractions_summary SEPARATOR '. ') AS infractions_summary,
               LISTAGG(infractions_summary, '. ') AS infractions_summary,
               SUM(num_significant) AS num_significant,
               SUM(num_crucial) AS num_crucial,
               SUM(num_minor) AS num_minor,
               SUM(num_na) AS num_na,
               SUM(num_infractions) AS num_infractions,
               {action_strs},
               {outcome_strs}
        FROM (
            SELECT establishment_id,
                   establishmenttype,
                   establishment_address,
                   inspection_date,
                   inspection_id,
                   -- GROUP_CONCAT(infraction_details SEPARATOR '. ') AS infractions_summary,
                   LISTAGG(infraction_details, '. ') AS infractions_summary,
                   establishment_status,
                   action,
                   court_outcome,
                   SUM(CASE WHEN severity LIKE '%%S - Significant' THEN 1 ELSE 0 END) AS num_significant,
                   SUM(CASE WHEN severity LIKE '%%C - Crucial' THEN 1 ELSE 0 END) AS num_crucial,
                   SUM(CASE WHEN severity LIKE '%%M - Minor' THEN 1 ELSE 0 END) AS num_minor,
                   SUM(CASE WHEN severity LIKE '%%NA -' THEN 1 ELSE 0 END) AS num_na,
                   COUNT(infraction_details) AS num_infractions
            FROM inspections
            WHERE establishmenttype IN {establishment_types_wanted_str}
            AND severity IS NULL
            OR severity IN ('S - Significant', 'C - Crucial', 'M - Minor')
            GROUP BY establishment_id, establishmenttype, establishment_address, inspection_date, inspection_id, establishment_status, action, court_outcome
        ) AS combo
        GROUP BY establishment_id,
                 establishmenttype,
                 establishment_address,
                 inspection_date,
                 inspection_id,
                 establishment_status
        """
df_query = show_sql_df(query, cur, None, True, True, False)
df_query = df_query.astype(dtypes_dict).astype(outcomes_dtypes_dict)
df_query

CPU times: user 2.28 s, sys: 65.5 ms, total: 2.35 s
Wall time: 3.85 s


,establishment_id,establishmenttype,establishment_address,inspection_date,inspection_id,establishment_status,infractions_summary,num_significant,num_crucial,num_minor,...,num_conviction_ordered_to_close_by_court,num_charges_withdrawn,num_pending,num_cancelled,num_conviction_suspended_sentence,num_conviction_fined,num_charges_dismissed,num_conviction_fined_order_to_close_by_court,num_null,num_conviction_probationary_order
0,10252788,Food Store (Convenience / Variety),1942 DANFORTH AVE,2015-06-11,103511714,Pass,Operator fail to properly maintain rooms. Oper...,0,0,20,...,0,0,0,0,0,0,0,0,1,0
1,10572803,Food Take Out,850 ADELAIDE ST W,2017-03-30,103939992,Pass,,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2,10609041,Cafeteria - Public Access,747 DON MILLS RD,2019-07-19,104519522,Pass,,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,10199628,Restaurant,2009 YONGE ST,2015-07-23,103537835,Pass,Operator fail to properly wash surfaces in roo...,6,0,6,...,0,0,0,0,0,0,0,0,2,0
4,10371510,Restaurant,415 PARLIAMENT ST,2015-08-05,103545405,Pass,FAIL TO HAVE TEST REAGENT AVAILABLE AT PLACE ...,10,0,0,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
213583,9404206,Restaurant,2293 ST CLAIR AVE W,2013-02-04,102918772,Pass,,0,0,0,...,0,0,0,0,0,0,0,0,1,0
213584,9421632,Food Store (Convenience / Variety),462 KINGSTON RD,2013-01-24,102905410,Pass,,0,0,0,...,0,0,0,0,0,0,0,0,1,0
213585,10285469,Restaurant,1221 MARKHAM RD,2011-09-15,102599543,Pass,,0,0,0,...,0,0,0,0,0,0,0,0,1,0
213586,10481321,Restaurant,382 YONGE ST,2017-06-02,103982464,Pass,OPERATE FOOD PREMISE - FAIL TO EQUIP FACILITY ...,3,0,0,...,0,0,0,0,0,0,0,0,1,0


Verify that no unwanted severity (`NA - ...`) is present in the query output

In [29]:
assert df_query[df_query["num_na"] > 0].empty

**Notes**
1. We are excluding unwanted establishments and infractions where the severity is not valid. We discussed both of these choices earlier in the **Select from the Different Types of Establishments that were Inspected** and **Types of Infraction Severities** sub-sections respectively.
2. The `establishment_status` column is included in order to have access to the status of the establishment after the inspection, should this be a useful predictor of the likelihood of future crucial / significant infractions. Each inspection must result in a single `establishment_status` and this will be verified later.

### Remove Inspections that took more than one day to complete

A single inspection should be completed on one day. It should not be spread out over more than one day. From the above aggregated result, get inspections (`inspection_id`s) that took more than one day to complete
- group by establishment and `inspection_id` and count the number of unique dates

In [30]:
df_query.groupby(
    ["establishment_id", "establishmenttype", "establishment_address", "inspection_id"],
    as_index=False,
)["inspection_date"].nunique().query("inspection_date > 1").sort_values(
    by=["inspection_date"], ascending=False
)

,establishment_id,establishmenttype,establishment_address,inspection_id,inspection_date
18873,9011494,Food Take Out,2677 KENNEDY RD,103420956,2
19836,9011956,Restaurant,57 OSSINGTON AVE,103455536,2
201414,10602403,Restaurant,15 WILLIAM KITCHEN RD,104013100,2
201411,10602331,Food Store (Convenience / Variety),30 DENTON AVE,104012758,2
200380,10595657,Food Store (Convenience / Variety),944 KINGSTON RD,103971961,2
198592,10588604,Restaurant,2516 FINCH AVE W,103925090,2
197179,10584091,Restaurant,744 WILSON AVE,103889199,2
192160,10568777,Restaurant,4 COLLIER ST,103769871,2
191145,10565743,Restaurant,986 BLOOR ST W,103748396,2
190538,10564048,Restaurant,120 CUMBERLAND ST,103736038,2


These inspections wlill need to be removed from the data. The reason for this occurrence is one of the followig
- the inspector needed to go back for a re-inspection
- the date was incorrectly entered
- unknown

It is reassuring that there are a small number of such inspections. Since we don't know the exact reason for this occurrence, we will remove all such inspections (that occurred on more than one day) from the data.

We will query this result to only get inspections that were completed on one day. We will do this using the establishment and inspection columns only, ignoring the counts and text column from earlier

In [31]:
%%time
df_query_no_multi_day_inspections = df_query.groupby(
    ["establishment_id", "establishmenttype", "establishment_address", "inspection_id"],
    as_index=False,
)["inspection_date"].nunique().query("inspection_date == 1").sort_values(
    by=["inspection_date"], ascending=False
)
df_query_no_multi_day_inspections

CPU times: user 247 ms, sys: 0 ns, total: 247 ms
Wall time: 246 ms


,establishment_id,establishmenttype,establishment_address,inspection_id,inspection_date
0,1222579,Food Take Out,870 MARKHAM RD,102810896,1
142431,10453199,Food Take Out,5160 YONGE ST,103078043,1
142355,10453113,Food Take Out,2600 WESTON RD,103013657,1
142356,10453113,Food Take Out,2600 WESTON RD,103034548,1
142357,10453113,Food Take Out,2600 WESTON RD,103198218,1
...,...,...,...,...,...
71185,10282520,Restaurant,2798 VICTORIA PARK AVE,103375091,1
71186,10282520,Restaurant,2798 VICTORIA PARK AVE,103428962,1
71187,10282520,Restaurant,2798 VICTORIA PARK AVE,103571685,1
71188,10282520,Restaurant,2798 VICTORIA PARK AVE,103730672,1


We'll now merge this result with the aggregated data from the SQL query in order to get all columns (including text and counts) for inspections that were completed on a single date

In [32]:
%%time
df = df_query_no_multi_day_inspections.drop(columns=["inspection_date"]).merge(df_query, on=["establishment_id", "establishmenttype", "establishment_address", "inspection_id"])
df["inspection_date"] = pd.to_datetime(df["inspection_date"])
df = df.sort_values(by=["establishment_id", "establishmenttype", "establishment_address", "inspection_id", "inspection_date"])
df

CPU times: user 358 ms, sys: 47.6 ms, total: 405 ms
Wall time: 405 ms


,establishment_id,establishmenttype,establishment_address,inspection_id,inspection_date,establishment_status,infractions_summary,num_significant,num_crucial,num_minor,...,num_conviction_ordered_to_close_by_court,num_charges_withdrawn,num_pending,num_cancelled,num_conviction_suspended_sentence,num_conviction_fined,num_charges_dismissed,num_conviction_fined_order_to_close_by_court,num_null,num_conviction_probationary_order
0,1222579,Food Take Out,870 MARKHAM RD,102810896,2012-08-21,Pass,,0,0,0,...,0,0,0,0,0,0,0,0,1,0
53384,1222579,Food Take Out,870 MARKHAM RD,103015258,2013-06-26,Conditional Pass,Food handler fail to wear headgear. Operator f...,4,4,8,...,0,0,0,0,0,0,0,0,2,0
142352,1222579,Food Take Out,870 MARKHAM RD,103015259,2013-06-27,Pass,,0,0,0,...,0,0,0,0,0,0,0,0,1,0
142353,1222579,Food Take Out,870 MARKHAM RD,103133558,2013-12-20,Pass,Food handler fail to wear headgear. Operator f...,0,0,6,...,0,0,0,0,0,0,0,0,1,0
142354,1222579,Food Take Out,870 MARKHAM RD,103329697,2014-09-09,Pass,Operator fail to properly maintain rooms. Oper...,3,0,12,...,0,0,0,0,0,0,0,0,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71183,10690616,Food Take Out,4698 YONGE ST,104594530,2019-10-23,Pass,,0,0,0,...,0,0,0,0,0,0,0,0,1,0
71184,10690642,Bake Shop,20 ST PATRICK ST,104594681,2019-10-23,Pass,FAIL TO PROVIDE THERMOMETER IN REFRIGERATION E...,1,0,0,...,0,0,0,0,0,0,0,0,1,0
71185,10690660,Restaurant,549 BLOOR ST W,104594800,2019-10-23,Pass,FAIL TO MAINTAIN HANDWASHING STATIONS (LIQUID ...,1,0,1,...,0,0,0,0,0,0,0,0,2,0
71186,10690679,Food Take Out,1175 ST CLAIR AVE W,104594954,2019-10-23,Pass,SANITIZE UTENSILS IN WATER FOR LESS THAN 45 SE...,1,0,0,...,0,0,0,0,0,0,0,0,1,0


As a sanity check, we will now count how many `inspection_id`s and `inspection_date`s occurred for each establishment. This will give us the number of inspections per establishment. We should get the same number of inspections if we count `inspection_id` or `inspection_date`. The result of this aggregation is shown below

In [33]:
recomp = (
    df.groupby(
        ["establishment_id", "establishmenttype", "establishment_address"],
        as_index=False,
    )
    .agg({"inspection_id": "count", "inspection_date": "count"})
    .rename(
        columns={
            "inspection_id": "num_inspection_ids",
            "inspection_date": "num_inspection_dates",
        }
    )
)
display(recomp.head())

,establishment_id,establishmenttype,establishment_address,num_inspection_ids,num_inspection_dates
0,1222579,Food Take Out,870 MARKHAM RD,14,14
1,1222580,Supermarket,1550 JANE ST,4,4
2,1222807,Restaurant,1635 LAWRENCE AVE W,22,22
3,1223056,Restaurant,606 BROWNS LINE,18,18
4,1223438,Food Take Out,500 REXDALE BLVD,10,10


As we can see below, the number of inspections per establishment is equal when calculated using `inspection_id` or `inspection_date`

In [34]:
assert recomp[recomp["num_inspection_ids"] != recomp["num_inspection_dates"]].empty

We can also verify that the sum (total) of the inspections adds up to the number of rows in the aggregated data (after removing the inspections that are spread across multiple dates), and this is shown below

In [35]:
assert recomp["num_inspection_ids"].sum() == len(df)
assert recomp["num_inspection_dates"].sum() == len(df)

### Remove Re-Inspections

Next, we need to eliminate inspections that occurred within two days of the initial inspection, since these follow-up inspections correspond to re-inspections (as mentioned earlier). Now, a single `inspection_id` corresponds to a single `inspection_date`. Starting with the above result, we will group by establishment and calculate difference in days between successive `inspection_date`s. This will give us the time gap between successive inspections of a single establishment. This time gap needs to be more than two days to avoid including re-inspections.

Get the time gap between successive inspections of a single establishment

In [36]:
%%time
df["days_to_next"] = (
    df.groupby(
        [
            "establishment_id",
            "establishmenttype",
            "establishment_address",
        ],
    )["inspection_date"]
    .diff(-1)
    .dt.days.abs()
)
df

CPU times: user 4.82 s, sys: 10.4 ms, total: 4.83 s
Wall time: 4.74 s


,establishment_id,establishmenttype,establishment_address,inspection_id,inspection_date,establishment_status,infractions_summary,num_significant,num_crucial,num_minor,...,num_charges_withdrawn,num_pending,num_cancelled,num_conviction_suspended_sentence,num_conviction_fined,num_charges_dismissed,num_conviction_fined_order_to_close_by_court,num_null,num_conviction_probationary_order,days_to_next
0,1222579,Food Take Out,870 MARKHAM RD,102810896,2012-08-21,Pass,,0,0,0,...,0,0,0,0,0,0,0,1,0,309.0
53384,1222579,Food Take Out,870 MARKHAM RD,103015258,2013-06-26,Conditional Pass,Food handler fail to wear headgear. Operator f...,4,4,8,...,0,0,0,0,0,0,0,2,0,1.0
142352,1222579,Food Take Out,870 MARKHAM RD,103015259,2013-06-27,Pass,,0,0,0,...,0,0,0,0,0,0,0,1,0,176.0
142353,1222579,Food Take Out,870 MARKHAM RD,103133558,2013-12-20,Pass,Food handler fail to wear headgear. Operator f...,0,0,6,...,0,0,0,0,0,0,0,1,0,263.0
142354,1222579,Food Take Out,870 MARKHAM RD,103329697,2014-09-09,Pass,Operator fail to properly maintain rooms. Oper...,3,0,12,...,0,0,0,0,0,0,0,2,0,121.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71183,10690616,Food Take Out,4698 YONGE ST,104594530,2019-10-23,Pass,,0,0,0,...,0,0,0,0,0,0,0,1,0,NaN
71184,10690642,Bake Shop,20 ST PATRICK ST,104594681,2019-10-23,Pass,FAIL TO PROVIDE THERMOMETER IN REFRIGERATION E...,1,0,0,...,0,0,0,0,0,0,0,1,0,NaN
71185,10690660,Restaurant,549 BLOOR ST W,104594800,2019-10-23,Pass,FAIL TO MAINTAIN HANDWASHING STATIONS (LIQUID ...,1,0,1,...,0,0,0,0,0,0,0,2,0,NaN
71186,10690679,Food Take Out,1175 ST CLAIR AVE W,104594954,2019-10-23,Pass,SANITIZE UTENSILS IN WATER FOR LESS THAN 45 SE...,1,0,0,...,0,0,0,0,0,0,0,1,0,NaN


Show all re-inspections (for this, the time gap will be two days or less)

In [37]:
df.query("days_to_next <= 2")

,establishment_id,establishmenttype,establishment_address,inspection_id,inspection_date,establishment_status,infractions_summary,num_significant,num_crucial,num_minor,...,num_charges_withdrawn,num_pending,num_cancelled,num_conviction_suspended_sentence,num_conviction_fined,num_charges_dismissed,num_conviction_fined_order_to_close_by_court,num_null,num_conviction_probationary_order,days_to_next
53384,1222579,Food Take Out,870 MARKHAM RD,103015258,2013-06-26,Conditional Pass,Food handler fail to wear headgear. Operator f...,4,4,8,...,0,0,0,0,0,0,0,2,0,1.0
142421,9000002,Food Take Out,361 OAKWOOD AVE,102611725,2011-10-05,Conditional Pass,Food handler fail to wear headgear. Operator f...,3,0,2,...,0,0,0,0,0,0,0,1,0,1.0
142397,9000026,Food Take Out,2372 EGLINTON AVE E,103930674,2017-03-15,Conditional Pass,Operator fail to provide accurate indicating t...,12,0,0,...,0,0,0,0,0,0,0,1,0,2.0
142249,9000029,Food Take Out,2548 EGLINTON AVE W,102594872,2011-09-07,Conditional Pass,Display hazardous foods at internal temperatur...,1,1,0,...,0,0,0,0,0,0,0,1,0,2.0
142268,9000031,Food Take Out,759 JANE ST,103484365,2015-05-07,Conditional Pass,Employee fail to wash hands when required. Emp...,0,6,0,...,0,0,0,0,0,0,0,1,0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71313,10687755,Bake Shop,262 DUPONT ST,104573182,2019-09-25,Conditional Pass,Fail to maintain records of pest control measu...,3,0,0,...,0,0,0,0,0,0,0,1,0,2.0
71141,10688507,Supermarket,28 NELSON ST,104578150,2019-10-02,Conditional Pass,FAIL TO MAINTAIN HANDWASHING STATIONS (LIQUID ...,2,2,0,...,0,0,0,0,0,0,0,2,0,2.0
71162,10689846,Restaurant,1106 BLOOR ST W,104589722,2019-10-16,Conditional Pass,FAIL TO MAINTAIN HANDWASHING STATIONS (LIQUID ...,2,1,0,...,0,0,0,0,0,0,0,1,0,1.0
71203,10690129,Restaurant,1571 SANDHURST CRCL,104591530,2019-10-21,Conditional Pass,Fail to protect food from contamination or adu...,2,1,0,...,0,0,0,0,0,0,0,1,0,2.0


We need to remove these re-inspections from the aggregated data from the end of the previous section.

There are some establishments that might have
- closed permanently after one inspection
- just opened, so have only been inspected once to-date

and these appear with a missing value in the `days_to_next` column. These establishments' inspections can be kept in the data, so below we will remove re-inspections (`days_to_next` < 2) and keep establishments with a single inspection (`days_to_next` has a missing value)

In [38]:
df = df.query("days_to_next > 2 | days_to_next.isna()").reset_index(drop=True).copy()
df

,establishment_id,establishmenttype,establishment_address,inspection_id,inspection_date,establishment_status,infractions_summary,num_significant,num_crucial,num_minor,...,num_charges_withdrawn,num_pending,num_cancelled,num_conviction_suspended_sentence,num_conviction_fined,num_charges_dismissed,num_conviction_fined_order_to_close_by_court,num_null,num_conviction_probationary_order,days_to_next
0,1222579,Food Take Out,870 MARKHAM RD,102810896,2012-08-21,Pass,,0,0,0,...,0,0,0,0,0,0,0,1,0,309.0
1,1222579,Food Take Out,870 MARKHAM RD,103015259,2013-06-27,Pass,,0,0,0,...,0,0,0,0,0,0,0,1,0,176.0
2,1222579,Food Take Out,870 MARKHAM RD,103133558,2013-12-20,Pass,Food handler fail to wear headgear. Operator f...,0,0,6,...,0,0,0,0,0,0,0,1,0,263.0
3,1222579,Food Take Out,870 MARKHAM RD,103329697,2014-09-09,Pass,Operator fail to properly maintain rooms. Oper...,3,0,12,...,0,0,0,0,0,0,0,2,0,121.0
4,1222579,Food Take Out,870 MARKHAM RD,103420091,2015-01-08,Pass,Operator fail to properly wash equipment. Oper...,3,0,6,...,0,0,0,0,0,0,0,2,0,713.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
205748,10690616,Food Take Out,4698 YONGE ST,104594530,2019-10-23,Pass,,0,0,0,...,0,0,0,0,0,0,0,1,0,NaN
205749,10690642,Bake Shop,20 ST PATRICK ST,104594681,2019-10-23,Pass,FAIL TO PROVIDE THERMOMETER IN REFRIGERATION E...,1,0,0,...,0,0,0,0,0,0,0,1,0,NaN
205750,10690660,Restaurant,549 BLOOR ST W,104594800,2019-10-23,Pass,FAIL TO MAINTAIN HANDWASHING STATIONS (LIQUID ...,1,0,1,...,0,0,0,0,0,0,0,2,0,NaN
205751,10690679,Food Take Out,1175 ST CLAIR AVE W,104594954,2019-10-23,Pass,SANITIZE UTENSILS IN WATER FOR LESS THAN 45 SE...,1,0,0,...,0,0,0,0,0,0,0,1,0,NaN


### Verify that Every Inspection has a Single Establishment Status

As a final sanity check, we will verify that a single inspection has only one `establishment_status`. Recall that the status is assigned to an establishment after the inspection has been performed. There can only be one status assigned to an establishment after the inspection. We will verify that every inspection has only a single `establishment_status`.

To do this, we will exclude `establishment_status` from the `.groupby()` (in the SQL query above, it was included in the `GROUP BY`) and aggregate (count) the number of unique values in this column for every inspection
- for every inspection, this count should be `1`

In [39]:
%%time
# Get number of unique `establishment_status` values for every inspection
num_statuses_per_inspection = (
    df.groupby(
        [
            "establishment_id",
            "establishmenttype",
            "establishment_address",
            "inspection_id",
            "inspection_date",
        ],
        as_index=False
    )["establishment_status"]
    .nunique()
)
# Verify that every inspection has only one `establishment_status`
assert num_statuses_per_inspection.query("establishment_status == 1").shape[0] == len(df)

CPU times: user 196 ms, sys: 0 ns, total: 196 ms
Wall time: 195 ms


As expected, there is only one status for every inspection.

With this, our aggregation (transformation) of the raw infractions data is completed. We now have a single row per inspection (at a single establishment, on a single date).

## Create the Class Labels column

We'll create a binary column to detect if a significant or crucial infraction (presenting a potential or immediate health hazard) was detected or not (minor or no infraction), since this is what our ML model will be trained to predict (see discussion at the end of the **Types of Infractions Severities** sub-section earlier)

In [40]:
mask = (df["num_significant"] > 0) | (df["num_crucial"] > 0)
df["is_infraction"] = 0
df.loc[mask, "is_infraction"] = 1

The class-imbalance is shown below (although we should not be interpreting this until we have split the data for ML experiments)

In [41]:
display(
    df["is_infraction"]
    .value_counts(normalize=True)
    .rename("fraction")
    .to_frame()
    .merge(
        df["is_infraction"].value_counts().rename("num_inspections").to_frame(),
        left_index=True,
        right_index=True,
        how="inner",
    )
)

,fraction,num_inspections
0,0.796951,163975
1,0.203049,41778


## Summary of Assumptions Made

1. Include establishment types that are similar to Restaurants or Grocery Stores, based on other cities ML implementations.
2. Exclude inspections with an invalid severity - `NA - ...`, since
   - we don't know the reason for this
   - a clear mapping between such inspections and the possible establishment status (Pass, Conditional Pass or Closed) is not possible
3. Remove re-inspections
4. Remove inspections that occurred over more than one day
5. When defining the ML class labels consider Significant and Crucial infractions together (minority class) since these present either a potential or immediate health hazard
   - by comparison, a Minor infraction presents a minimal health risk
6. The following inspections are kept in the data
   - inspections for establishments that have not been previously inspected
   - inspections for establishments that have closed since their last inspection

## Export Transformed Data to CSV

We'll now export this transformed data (aggregated by inspection and filtered to remove (a) inspections that took more than one day to complete, (b) re-inspections, (c) unwanted establishment types and (d) inspections with an invalid severity) to a CSV file which can be loaded into Python for further processing

In [42]:
%%time
time_now  = datetime.now().strftime('%Y%m%d_%H%M%S')
df.drop(columns=["days_to_next"]).to_csv(
    f"data/processed/{transformed_fname_prefix}__{time_now}.csv",
    index=False,
)

CPU times: user 1.85 s, sys: 77.5 ms, total: 1.93 s
Wall time: 1.93 s


## Disconnect from MySQL Database

In [43]:
cur.close()
conn.close()